[![imagenes/pythonista.png](imagenes/pythonista.png)](https://pythonista.io)

## La biblioteca *SQLAlchemy*.

[*SQLAlchemy*](http://www.sqlalchemy.org/) comprende diversas herramientas enfocadas a interactuar con bases de datos relacionales de forma "pythonica".

Consta de:
* **SQLAlchemy Core**, la cual permite crear una interfaz genérica e independiente del gestor de base de datos por medio de un lenguaje  de expresiones basado en SQL.
* **SQLAlchemy ORM** Mapeador entre objetos y transacciones relacionales u ORM (por las sigas de object-relational mapper).

Conocer a profundidad la biblioteca de SQLAlchemy podría extenderse por varios capítulos, por lo que para fines de este curso, solo se explorarán superficialmente el ORM de SQLAlchemy y la extensión de Jupyter del lenguaje de expresiones de SQLAlchemy ([*ipython-sql*](https://github.com/catherinedevlin/ipython-sql)), la cual aprovecha las funcionalidades de SQLAlchemy Core. 

**Nota:** En este ejemplo tulizaremos el gestor de bases de datos [SQLite](https://sqlite.org/index.html), el cual vienejunto con Python y es capaz de crear bases de datos en un archivo o incluso en memoria.

In [ ]:
!pip install sqlalchemy

In [ ]:
import sqlalchemy

## Ruta de conexión a un gestor de la base de datos.

El primer paso para poder interactuar con una base de datos es conectándose a esta. Para ello es necesario conocer la ruta y contar con las credenciales (generalmente usuario y contraseña) para poder acceder.

La sintaxis de la ruta de conexión a una base de datos utiliza la siguiente sintaxis.

```
<<dialecto><controlador>://<usuario>:<contraseña>@<ruta del servidor>:<puerto>/<base de datos>
```

Para mayor información, consultar: 

http://docs.sqlalchemy.org/en/latest/core/engines.html#database-urls

## La clase *sqlalchemy.engine.base.Engine*.

La clase *sqlalchemy.engine.base.Engine* tiene por finalidad instanciar objetos que representan el elemento fundamental de esta biblioteca para conectarse con una base de datos y a su vez mapear los atributos de los objetos creados a partir del modelo ORM.

Para instanciar un objeto de *sqlalchemy.engine.base.Engine* se utliliza la función *sqlalchemy.create_engine()* con la siguiente sintaxis:

```
create_engine('<ruta>', <argumentos>)
````

**Ejemplo:**

Se instanciará un objetos a partir de *sqlalchemy.Engine* conectado a una base de datos localizada en [*data/alumnos.db*](data/alumnos.txt)

In [ ]:
engine = sqlalchemy.create_engine('sqlite:///data/alumnos.db')

In [ ]:
type(engine)

## Definición de un modelo objeto-relacional.

Antes de crear la base de datos es necesario definir un modelo que mapee a un objeto con al menos una tabla en la base de datos.

La función *sqlalchemy.ext.declarative.declaratrive_base()* nos permite crear un modelo a partir de las subclases de *sqlalchemy.ext.declarative.api.DeclarativeMeta*.

**Ejemplo:**

Se creará la clase *Base* como superclase genera.

**Nota:** A partir de este momento cuando se utilice el nombre *Base* se referirá a una subclase de  *sqlalchemy.ext.declarative.api.DeclarativeMeta*.

In [ ]:
from sqlalchemy.ext.declarative import declarative_base

In [ ]:
Base = declarative_base()

In [ ]:
type(Base)

### Definición de columnas de una tabla como atributo de una clase.

Las subclase de *Base* corresponden a tablas dentro de una base de datos.

Estas subclases tiene el atributo *\_\_tablename\_\_* que corresponde al nombre de la tabla a la cual está mapeando sus atributos.

La sintaxis es la siguiente:
```
class <Nombre de la Clase>(Base):
    __tablename__ = <nombre de la tabla>
    ...
    ...
    ...
```

Cada objeto instanciado de estas subclases corresponderá a un registro en la tabla definida en *\_\_tablename\_\_*.

#### Definición de columnas como atributos.

Para mapear un atribulo a una columna de la tabla se utiliza la clase *sqlalchemy.Column* con la siguiente sintaxis.

```
class <Nombre de la Clase>(Base):
    __tablename__ = <nombre de la tabla>
    <nombre del atributo> = sqlalchemy.Column(<tipo de dato>, <argumentos>)
    ...
    ...
```

##### Algunos parámetros relevantes de *sqlalchemy.Column*. 

* El parámetro *primary\_key* con argumento igual a *True* indica que dicho atributo  y será la clave primaria de la tabla definida por db.Model.\_\_table_name\_\_ y además será utilizado como parámetro al instanciar a la subclase de *db.Model*.
* El parámetro *unique* con argumento igual a *True* indica que no puede haber dos valores idénticos en la columna. 

##### Tipos de datos de SQLAlchemy.

Cada columna debe de ser definida en la base de datos con un tipo de dato específico.

Los tipos de datos que SQLAlchemy acepta pueden ser consultados en http://docs.sqlalchemy.org/en/latest/core/type_basics.html.



En este ejemplo utilizaremos los tipos:

* [*Integer*](http://docs.sqlalchemy.org/en/latest/core/type_basics.html#sqlalchemy.types.Integer).
* [*String*](http://docs.sqlalchemy.org/en/latest/core/type_basics.html#sqlalchemy.types.String).
* [*Float*](http://docs.sqlalchemy.org/en/latest/core/type_basics.html#sqlalchemy.types.Float).
* [*Boolean*](http://docs.sqlalchemy.org/en/latest/core/type_basics.html#sqlalchemy.types.Boolean).

A continuación se creará la clase *Alumno* que es subclase de *Base*, la cual estará ligada a la tabla *alumnos* cuyas columnas/atributos son:

* *cuenta* de tipo *Integer* con el argumento *primary\_key=True*.
* *nombre* de tipo *String* con tamaño de 50 caracteres.
* *primer_apellido* de tipo *String* con tamaño de 50 caracteres.
* *segundo_apellido* de tipo *String* con tamaño de 50 caracteres.
* *carrera* de tipo *String* con tamaño de 50 caracteres.
* *semestre* de tipo *Integer*.
* *promedio* de tipo *Float*.
* *al_corriente* de tipo *Boolean*. 

In [ ]:
from sqlalchemy import Integer, String, Float, Boolean, Column

In [ ]:
class Alumno(Base):
    __tablename__ = 'alumnos'
    cuenta = Column(Integer, primary_key=True)
    nombre = Column(String(50))
    primer_apellido = Column(String(50))
    segundo_apellido = Column(String(50))
    carrera = Column(String(50))
    semestre = Column(Integer)
    promedio = Column(Float)
    al_corriente = Column(Boolean)

### Creación de las tablas de la base de datos con el método *Base.metadata.create_all()*.

Una base de datos puede consistir de múltiples tablas. En este caso, la base de datos sólo contendrá a la tabla *alumnos* ligadas a la clase *Alumnos*.

Para crear la base de datos con las tablas definidas se utiliza el método *Base.metadata.create_all()* en la base de datos gestionada por el objeto instanciado de *sqlalchemy.engine.base.Engine*. 

* En caso de que no exista el archivo con la base de datos, este será creado.
* En caso de que ya existan tablas definidas en la base de datos, sólos e crearán las que sean nuevas y los datos que ya contienen no serán eliminados.

In [ ]:
Base.metadata.create_all(engine)

Ahora se creó el archivo *alumnos.db* en el directorio [*data*](data/).

A partir de este momento, cada objeto instanciado de *Alumno* puede ser representado como un registro de la tabla *alumnos*.

### Creación de una sesión.

La función *sqlalchemy.orm.sessionmaker()* permite crear una clase *sqlalchemy.orm.session.sessionmaker* que contiene atributos y métodos que permiten interactuar con la base de datos. En este y el próximo capítulo usaremos:

* El método *add()* que añade o sustituye el registro ligado al objeto instanciado de una subclase de *Base* en el registro correspondiente dentro de la base de datos.
* El método *delete()* que elimina el registro ligado al objeto.
* El método *commit()* el cual aplica los cambios en la base de datos.

In [ ]:
from sqlalchemy.orm import sessionmaker
Sesion = sessionmaker(bind=engine)

In [ ]:
type(Sesion)

In [ ]:
sesion = Sesion()

###  Poblamiento de la base de datos a partir de un archivo.

El archivo [*data/alumnos.txt*](data/alumnos.txt) contiene la representación de un objeto tipo *list* que a su vez contiene objetos tipo *dir* con los campos:

* *'Cuenta'*.
* *'Nombre'*.
* *'Primer Apellido'*.
* *'Segundo Apellido'*.
* *'Carrera'*.
* *'Semestre'*.
* *'Promedio'*.
* *'Al Corriente'*.

Se puede observar que los identificadores de los campos son similares a los nombres de las columnas definidas en la clase *Alumnos*. Sólo es necesario convertirlos a minúsculas y sustituir los espacios con guiones bajos.

La siguiente celda convertirá cada objeto tipo *dict* del archivo *data/alumnos.txt* en un objeto instaciado de *Alumno* y lo ligará a la base de datos.

Cada objeto ligado a *alumno* será sustituido por el siguiente cada vez que *Alumno* sea instanciado en cada iteración. Esto implica que el objeto sustituído será destruido, pero sus atributos persistirán en la base de datos.

In [ ]:
# Craga los datos del archivo.
with open('data/alumnos.txt', 'tr') as archivo:
    base = eval(archivo.read())
    
# Puebla la tabla alumno instanciando de Alumno a cada elemento de tipo _dict_.
for registro in base:
    # Se instancia el objeto alumno a partir de la clase Alumno.
    alumno  = Alumno(cuenta=registro['Cuenta'])
    del registro['Cuenta']
    for campo in registro:
        # Los campos son convertidos en atributos. 
        setattr(alumno, campo.lower().replace(' ', '_'), registro[campo])
    #
    sesion.add(alumno)
# Los cambios se aplican a la base de datos.
sesion.commit()

### Consultas mediante el ORM.

Los objetos de  subclases de *sqlalchemy.orm.session.sessionmaker* pueden realizar búsquedas relacionadas a los objetos instanciados de *Base* a partir de ellas y de las tablas ligadas a ellas.

SQLAlchemy puede realizar consultas tanto mediante álgebra relacional como de búsqueda sobre los atributos de sus objetos instanciados.

**Nota:** Para ilustrar estas operaciones se utilizará la clase *Alumno*, la cual es subclase de *Base*.


Para efectos de este curso sólo se explorarán los siguientes métodos:

* *query.first()* regresa al primer objeto encontrado en una búsqueda.
* *query.all()* regresa un objeto tipo *list* con todos los objetos resultantes de una búsqueda.
* *query.filter()* regresa un objeto de tipo *Query* con los objetos encontrados al ejecutar una búsqueda que satisfaga la expresión lógica sobre los atributos de la clase, la cual es ingresada como argumento.
* *query.filter_by()* regresa un objeto de tipo *Query* con los objetos encontrados al ejecutar una búsqueda en la tabla de la base de datos cuyos valores en la columna sean iguales al valor que se ingresa como argumento en la forma ```<columna>=<valor>```.

In [ ]:
sesion.query(Alumno).filter(Alumno.cuenta)

In [ ]:
sesion.query(Alumno).filter(Alumno.cuenta).all()

In [ ]:
sesion.query(Alumno).filter(Alumno.cuenta > 1231221).all()

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2018.</p>